In [1]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import streamlit as st
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from collections import defaultdict
from scipy.spatial.distance import cdist
import time
cid = 'YOUR CLIENT ID'
secret = 'YOUR CLIENT SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [6]:
df = pd.read_csv('fourtet.csv')
df.head()

,Unnamed: 0,track_name,album,artist_name,release_date,duration_ms,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0,Se Rompen,Se Rompen,Waleed,2021-08-06,322418,28,0.808,0.1800,0.719,0.691000,0.1070,-9.671,0.1620,130.997,0.359
1,1,Chameleon,Chameleon,Anthony Naples,2021-09-03,299821,33,0.617,0.0047,0.618,0.675000,0.2810,-11.498,0.0516,149.974,0.301
2,2,Echoes - Radio Edit 1,Echoes (Anniversary Edition),Directions,2021-09-03,278622,20,0.676,0.3780,0.609,0.933000,0.1020,-9.195,0.0320,92.542,0.784
3,3,Breathe Me - Four Tet Remix,Colour The Small One (Deluxe Edition),Sia,2004-01-01,301106,29,0.836,0.0694,0.478,0.031800,0.0823,-9.978,0.0789,120.005,0.633
4,4,Main Akeli Raat Jawan,Purana Mandir (Original Motion Picture Soundtr...,Ajit Singh,1984-01-01,327000,22,0.694,0.4230,0.721,0.000066,0.1460,-5.836,0.0595,111.381,0.853


In [7]:
df.isna().sum()

Unnamed: 0          0
track_name          0
album               0
artist_name         0
release_date        0
duration_ms         0
popularity          0
danceability        0
acousticness        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
dtype: int64

In [8]:
df.describe()

,Unnamed: 0,duration_ms,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,1723.000000,1.723000e+03,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000,1723.000000
mean,861.000000,3.032167e+05,13.202554,0.572201,0.422339,0.536662,0.503677,0.163398,-12.553478,0.097519,119.266135,0.427589
std,497.531574,1.794707e+05,14.628388,0.202897,0.368885,0.256796,0.389506,0.137327,5.642311,0.109484,27.455510,0.274762
min,0.000000,1.895700e+04,0.000000,0.000000,0.000002,0.000281,0.000000,0.020600,-37.114000,0.000000,0.000000,0.000000
25%,430.500000,1.925490e+05,0.000000,0.425000,0.047300,0.344500,0.018750,0.090150,-15.278500,0.040200,99.931500,0.180000
50%,861.000000,2.713340e+05,8.000000,0.594000,0.334000,0.562000,0.656000,0.111000,-11.326000,0.056200,121.008000,0.398000
75%,1291.500000,3.691595e+05,22.000000,0.743000,0.805000,0.747000,0.880000,0.172000,-8.508500,0.099950,133.994500,0.662000
max,1722.000000,2.241200e+06,69.000000,0.980000,0.996000,0.999000,0.984000,0.993000,-1.987000,0.947000,210.029000,0.996000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1723 non-null   int64  
 1   track_name        1723 non-null   object 
 2   album             1723 non-null   object 
 3   artist_name       1723 non-null   object 
 4   release_date      1723 non-null   object 
 5   duration_ms       1723 non-null   int64  
 6   popularity        1723 non-null   int64  
 7   danceability      1723 non-null   float64
 8   acousticness      1723 non-null   float64
 9   energy            1723 non-null   float64
 10  instrumentalness  1723 non-null   float64
 11  liveness          1723 non-null   float64
 12  loudness          1723 non-null   float64
 13  speechiness       1723 non-null   float64
 14  tempo             1723 non-null   float64
 15  valence           1723 non-null   float64
dtypes: float64(9), int64(3), object(4)
memory 

In [11]:
df = df.drop(['popularity', 'Unnamed: 0'], axis=1)
df

,track_name,album,artist_name,release_date,duration_ms,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Se Rompen,Se Rompen,Waleed,2021-08-06,322418,0.808,0.18000,0.71900,0.691000,0.1070,-9.671,0.1620,130.997,0.359
1,Chameleon,Chameleon,Anthony Naples,2021-09-03,299821,0.617,0.00470,0.61800,0.675000,0.2810,-11.498,0.0516,149.974,0.301
2,Echoes - Radio Edit 1,Echoes (Anniversary Edition),Directions,2021-09-03,278622,0.676,0.37800,0.60900,0.933000,0.1020,-9.195,0.0320,92.542,0.784
3,Breathe Me - Four Tet Remix,Colour The Small One (Deluxe Edition),Sia,2004-01-01,301106,0.836,0.06940,0.47800,0.031800,0.0823,-9.978,0.0789,120.005,0.633
4,Main Akeli Raat Jawan,Purana Mandir (Original Motion Picture Soundtr...,Ajit Singh,1984-01-01,327000,0.694,0.42300,0.72100,0.000066,0.1460,-5.836,0.0595,111.381,0.853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,Es war einmal,Sowiesoso,Cluster,1976,325400,0.643,0.75800,0.15600,0.976000,0.1040,-20.074,0.0419,117.372,0.314
1719,Kinoshita,Eph Reissue,Fridge,2002-05-07,302693,0.623,0.13200,0.94500,0.895000,0.3020,-5.775,0.0984,139.971,0.611
1720,Promises Of Fertility,For Those Of You Who Have Never (And Also Thos...,Huerco S.,2016-06-10,415933,0.260,0.82100,0.28400,0.841000,0.1100,-19.656,0.0367,61.829,0.425
1721,3,Get Out,Pita,2008-05-12,678080,0.228,0.00869,0.61900,0.928000,0.1540,-7.414,0.0483,105.538,0.207


In [12]:
X = df.select_dtypes(np.number)
X

,duration_ms,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,322418,0.808,0.18000,0.71900,0.691000,0.1070,-9.671,0.1620,130.997,0.359
1,299821,0.617,0.00470,0.61800,0.675000,0.2810,-11.498,0.0516,149.974,0.301
2,278622,0.676,0.37800,0.60900,0.933000,0.1020,-9.195,0.0320,92.542,0.784
3,301106,0.836,0.06940,0.47800,0.031800,0.0823,-9.978,0.0789,120.005,0.633
4,327000,0.694,0.42300,0.72100,0.000066,0.1460,-5.836,0.0595,111.381,0.853
...,...,...,...,...,...,...,...,...,...,...
1718,325400,0.643,0.75800,0.15600,0.976000,0.1040,-20.074,0.0419,117.372,0.314
1719,302693,0.623,0.13200,0.94500,0.895000,0.3020,-5.775,0.0984,139.971,0.611
1720,415933,0.260,0.82100,0.28400,0.841000,0.1100,-19.656,0.0367,61.829,0.425
1721,678080,0.228,0.00869,0.61900,0.928000,0.1540,-7.414,0.0483,105.538,0.207


In [17]:
cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=5, verbose=2))])
cluster_pipeline.fit(X)

Initialization complete
Iteration 0, inertia 14910.623112987589
Iteration 1, inertia 11074.426346062053
Iteration 2, inertia 10981.598807000095
Iteration 3, inertia 10935.874233929911
Iteration 4, inertia 10894.241182799975
Iteration 5, inertia 10834.485175077107
Iteration 6, inertia 10775.991173183742
Iteration 7, inertia 10721.87017032047
Iteration 8, inertia 10662.221918681742
Iteration 9, inertia 10611.828630659675
Iteration 10, inertia 10569.027421107521
Iteration 11, inertia 10533.264768476723
Iteration 12, inertia 10511.778014809965
Iteration 13, inertia 10487.949038732808
Iteration 14, inertia 10474.607892242706
Iteration 15, inertia 10459.751005134174
Iteration 16, inertia 10449.29482693587
Iteration 17, inertia 10440.987775836447
Iteration 18, inertia 10436.251008983247
Iteration 19, inertia 10433.096819606926
Iteration 20, inertia 10431.596328463203
Iteration 21, inertia 10430.804548346192
Iteration 22, inertia 10430.615457915266
Iteration 23, inertia 10430.524953992875
Iter

Pipeline(steps=[('scaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=5, verbose=2))])

In [18]:
cluster_labels = cluster_pipeline.predict(X)
df['cluster'] = cluster_labels
df['cluster'].value_counts()

4    567
2    486
0    419
1    133
3    118
Name: cluster, dtype: int64

In [19]:
pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = df['track_name']
projection['cluster'] = df['cluster']
projection

,x,y,title,cluster
0,-1.380372,0.650095,Se Rompen,4
1,-0.843733,0.855701,Chameleon,4
2,-0.659132,0.375345,Echoes - Radio Edit 1,4
3,-1.371654,-0.527855,Breathe Me - Four Tet Remix,0
4,-1.671719,-0.999178,Main Akeli Raat Jawan,0
...,...,...,...,...
1718,2.008769,0.846330,Es war einmal,2
1719,-2.089659,0.632492,Kinoshita,4
1720,2.737391,0.173636,Promises Of Fertility,2
1721,-0.025665,2.015595,3,4


In [20]:
fig = px.scatter(projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()